# Stage 2: Labeled Scenarios — Coverage Mapping for Your AI

---

## 🎯 What You'll Learn

By the end of this walkthrough, you'll understand:

1. **What labeled scenarios are** and how they differ from golden sets
2. **Why coverage mapping matters** for AI evaluation
3. **How to organize test cases** by category, complexity, and difficulty
4. **How to identify testing gaps** and prioritize improvements

---

## 📚 Understanding Labeled Scenarios

### What Are Labeled Scenarios?

While golden sets answer "Does it work?", labeled scenarios answer **"Does it work for all types of queries?"**

**Labeled scenarios are categorized test cases** that give you visibility into which parts of your system are well-tested and which need attention.

| Golden Sets | Labeled Scenarios |
|-------------|-------------------|
| 10-50 curated cases | 30-100+ categorized cases |
| Must all pass | Some failure is expected |
| Run on every commit | Run on releases |
| Catch regressions | Map coverage gaps |
| "Is it broken?" | "What's not tested?" |

### Why You Need This

Imagine your golden sets all pass. Great! But are you testing:
- ❓ All types of queries users actually ask?
- ❓ Edge cases and ambiguous questions?
- ❓ Multi-tool queries that require orchestration?
- ❓ Different difficulty levels?

**Without labeled scenarios, you have blind spots you don't even know about.**

---

## 🏗️ Where Labeled Scenarios Fit: The Eval Maturity Model

```
┌─────────────────────────────────────────────────────────────────┐
│                    EVAL FRAMEWORK MATURITY                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Stage 5: Experiments      ← Compare configurations            │
│      ▲                                                          │
│  Stage 4: Rubrics          ← Multi-dimensional scoring         │
│      ▲                                                          │
│  Stage 3: Replay Harnesses ← Reproducibility + rich metrics    │
│      ▲                                                          │
│  ★ Stage 2: LABELED SCENARIOS ★ ← Coverage mapping (YOU ARE HERE)│
│      ▲                                                          │
│  Stage 1: Golden Sets      ← Baseline correctness (completed)  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

Labeled scenarios **build on golden sets** by adding structure and categorization. You still run the same types of checks, but now you can see patterns in where your system succeeds and fails.

## 🔍 Behind the Scenes: How Labeling Works

Scenarios are organized in a **hierarchical structure** with three dimensions:

```
┌──────────────────────────────────────────────────────────────────────┐
│                    SCENARIO ORGANIZATION                              │
├──────────────────────────────────────────────────────────────────────┤
│                                                                       │
│  DIMENSION 1: TOOL TYPE                                               │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │ • vector_only  → Document search queries                    │     │
│  │ • sql_only     → Database/metrics queries                   │     │
│  │ • jira_only    → Ticket lookup queries                      │     │
│  │ • slack_only   → Message search queries                     │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                                                                       │
│  DIMENSION 2: COMPLEXITY                                              │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │ • single_tool  → Requires exactly one tool                  │     │
│  │ • multi_tool   → Requires 2+ tools working together         │     │
│  │ • synthesis    → Requires reasoning across sources          │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                                                                       │
│  DIMENSION 3: DIFFICULTY                                              │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │ • straightforward → Clear, direct questions                 │     │
│  │ • ambiguous       → Could go multiple ways                  │     │
│  │ • edge_case       → Unusual or tricky inputs                │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                                                                       │
└──────────────────────────────────────────────────────────────────────┘
```

This structure lets you answer questions like:
- "How well do we handle SQL queries?" → Filter by `sql_only`
- "Do multi-tool queries work?" → Filter by `multi_tool`
- "How do we handle edge cases?" → Filter by `edge_case`

In [1]:
import sys
sys.path.insert(0, "../setup_agent")

from evaluator import load_scenarios, flatten_scenarios, run_scenario

## 💻 Hands-On: Exploring the Scenario Structure

Let's load the scenarios and see how they're organized. Notice the hierarchical structure that enables filtering and coverage analysis:

In [2]:
# Load scenarios
scenarios = load_scenarios()

# Show the structure
print("Scenario Categories:")
print("-" * 40)
for category, subcategories in scenarios.items():
    print(f"\n{category}/")
    if isinstance(subcategories, dict):
        for subcategory, items in subcategories.items():
            count = len(items) if isinstance(items, list) else 0
            print(f"  └── {subcategory}: {count} scenarios")

Scenario Categories:
----------------------------------------

single_tool/
  └── vector_only: 6 scenarios
  └── sql_only: 5 scenarios
  └── jira_only: 4 scenarios
  └── slack_only: 4 scenarios

multi_tool/
  └── vector_and_sql: 2 scenarios
  └── vector_and_jira: 1 scenarios
  └── sql_and_slack: 1 scenarios
  └── jira_and_slack: 1 scenarios

edge_cases/
  └── empty_results: 1 scenarios
  └── off_topic: 1 scenarios
  └── typos: 1 scenarios
  └── very_long: 1 scenarios


## 🔎 Filtering Scenarios

One of the key benefits of labeled scenarios is **targeted testing**. You can run specific subsets to investigate issues or verify fixes:

| Filter | Use Case |
|--------|----------|
| `category_filter="single_tool"` | Test all single-tool queries |
| `subcategory_filter="sql_only"` | Focus on SQL queries |
| `difficulty_filter="edge_case"` | Check edge case handling |

Let's see filtering in action:

In [3]:
# Get all scenarios
all_scenarios = flatten_scenarios(scenarios)
print(f"Total scenarios: {len(all_scenarios)}")

# Filter to single_tool only
single_tool = flatten_scenarios(scenarios, category_filter="single_tool")
print(f"Single-tool scenarios: {len(single_tool)}")

# Filter to sql_only specifically
sql_only = flatten_scenarios(scenarios, subcategory_filter="sql_only")
print(f"SQL-only scenarios: {len(sql_only)}")

# Filter by difficulty
edge_cases = flatten_scenarios(scenarios, difficulty_filter="edge_case")
print(f"Edge case scenarios: {len(edge_cases)}")

Total scenarios: 28
Single-tool scenarios: 19
SQL-only scenarios: 5
Edge case scenarios: 4


## 🧪 Running a Single Scenario

Running scenarios works similarly to golden sets, but with added category context. Notice how the result includes metadata about category and difficulty:

In [4]:
# Run a single scenario
category, subcategory, scenario = all_scenarios[0]

print(f"Running: {scenario['id']}")
print(f"Category: {category}/{subcategory}")
print(f"Query: {scenario['query']}")
print(f"Difficulty: {scenario.get('difficulty', 'unknown')}")
print()

result = run_scenario(category, subcategory, scenario)

print(f"Passed: {result.passed}")
print(f"Tools used: {result.tools_used}")
print(f"Tool check: {result.tool_check}")
print(f"Content check: {result.content_check}")

if result.errors:
    print(f"Errors: {result.errors}")

Running: sc-v-001
Category: single_tool/vector_only
Query: What is our remote work policy?
Difficulty: straightforward

Passed: True
Tools used: ['vector_search']
Tool check: True
Content check: True


## 🏃 Running a Category

Targeted testing is powerful. If SQL queries seem broken, you can run just the SQL scenarios to investigate. This is much faster than running everything:

In [5]:
# Run all vector_only scenarios
vector_scenarios = flatten_scenarios(scenarios, subcategory_filter="vector_only")

print(f"Running {len(vector_scenarios)} vector_only scenarios:")
print("-" * 40)

results = []
for cat, subcat, scenario in vector_scenarios[:3]:  # Just first 3 for demo
    print(f"  {scenario['id']}: {scenario['query'][:40]}...", end=" ")
    result = run_scenario(cat, subcat, scenario)
    results.append(result)
    print("✓" if result.passed else "✗")

passed = sum(1 for r in results if r.passed)
print(f"\nResults: {passed}/{len(results)}")

Running 6 vector_only scenarios:
----------------------------------------
  sc-v-001: What is our remote work policy?... ✓
  sc-v-002: How much is the home office stipend?... ✓
  sc-v-003: What is our PTO policy?... ✓

Results: 3/3


## 📊 Coverage Analysis: The Real Power of Labels

**This is the key benefit of labeled scenarios**: You can see at a glance where you're well-covered and where you have gaps.

The coverage matrix shows:
- **Category breakdown** — How each type performs
- **Difficulty breakdown** — Easier vs harder queries
- **Visual indicators** — Spot problems quickly

This is how you answer: "What should we test next?"

In [6]:
# Count scenarios by category
coverage = {}
for cat, subcat, scenario in all_scenarios:
    key = f"{cat}/{subcat}"
    coverage.setdefault(key, {"total": 0, "by_difficulty": {}})
    coverage[key]["total"] += 1
    
    diff = scenario.get("difficulty", "unknown")
    coverage[key]["by_difficulty"].setdefault(diff, 0)
    coverage[key]["by_difficulty"][diff] += 1

print("Coverage Matrix:")
print("-" * 60)
print(f"{'Category':<35} {'Total':<8} {'Easy':<8} {'Ambig':<8} {'Edge':<8}")
print("-" * 60)

for key in sorted(coverage.keys()):
    data = coverage[key]
    easy = data["by_difficulty"].get("straightforward", 0)
    ambig = data["by_difficulty"].get("ambiguous", 0)
    edge = data["by_difficulty"].get("edge_case", 0)
    print(f"{key:<35} {data['total']:<8} {easy:<8} {ambig:<8} {edge:<8}")

Coverage Matrix:
------------------------------------------------------------
Category                            Total    Easy     Ambig    Edge    
------------------------------------------------------------
edge_cases/empty_results            1        0        0        1       
edge_cases/off_topic                1        0        0        1       
edge_cases/typos                    1        0        0        1       
edge_cases/very_long                1        0        0        1       
multi_tool/jira_and_slack           1        1        0        0       
multi_tool/sql_and_slack            1        1        0        0       
multi_tool/vector_and_jira          1        1        0        0       
multi_tool/vector_and_sql           2        2        0        0       
single_tool/jira_only               4        3        1        0       
single_tool/slack_only              4        3        1        0       
single_tool/sql_only                5        4        1        0     

## 📈 Interpreting the Coverage Matrix

When you see results like this:

```
                    | Pass Rate | Easy | Ambig | Edge |
--------------------|-----------|------|-------|------|
single_tool/vector  |    100%   |  3/3 |  1/1  |  1/1 |
single_tool/sql     |     67%   |  2/3 |  0/1  |  0/1 |  ⚠️ Gap!
multi_tool          |     50%   |  1/2 |  0/0  |  0/0 |  ⚠️ Gap!
```

**Action items become clear:**
1. SQL queries need improvement (only 67% passing)
2. Multi-tool orchestration is weak (only 50% passing)
3. We need more ambiguous and edge case tests for multi-tool

---

## 🎓 Key Takeaways

1. **Labels enable visibility** — You can't improve what you can't measure
2. **Hierarchical organization** — Category → Subcategory → Difficulty
3. **Targeted testing** — Run specific subsets when debugging
4. **Coverage gaps** — The matrix shows where to focus next
5. **Progressive enhancement** — Add scenarios as you find bugs

---

## ⏭️ What's Next?

Labeled scenarios give you coverage visibility, but there's still a problem: **every run produces slightly different results**.

- LLM responses vary due to temperature
- External services may behave differently
- Tests aren't fully reproducible

**Stage 3: Replay Harnesses** solves this with deterministic replay and richer metrics:

```bash
cd ../stage_3_replay_harnesses
uv run jupyter notebook walkthrough.ipynb
```